In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("master_district_month_clean.csv")

# Parse month safely
df["month"] = pd.to_datetime(df["month"])

# Sort (critical for lags later)
df = df.sort_values(["district_name", "month"]).reset_index(drop=True)


In [ ]:
agg_rules = {
    # STATIC
    "state_name": "first",
    "year": "first",
    "month_num": "first",
    "num_atms": "first",

    # SOCIO-ECONOMIC (slow moving)
    "population_total_k": "first",
    "working_age_pct": "first",
    "literacy_rate": "first",
    "median_income_k": "first",
    "business_activity_index": "mean",
    "cash_intensity_index": "mean",

    # ATM MIX
    "metro_atm_share": "mean",
    "urban_atm_share": "mean",
    "rural_atm_share": "mean",

    # NIGHT LIGHTS
    "ntl_index_mean": "mean",
    "ntl_index_latest": "mean",

    # NATIONAL / DIGITAL
    "withdrawal_volume_million": "mean",
    "withdrawal_value_crore": "mean",
    "upi_txn_volume_mn": "mean",
    "pos_card_txn_volume_mn": "mean",
    "wallet_txn_volume_mn": "mean",
    "digital_txn_volume_mn": "mean",
    "digital_share_txn": "mean",
    "atm_share_txn": "mean",

    # TARGETS (SUM!)
    "district_atm_withdrawal_volume_mn": "sum",
    "district_atm_withdrawal_value_cr": "sum",
}

df = (
    df
    .groupby(["district_name", "month"], as_index=False)
    .agg(agg_rules)
)


In [ ]:
# No duplicates now
assert df.duplicated(["district_name", "month"]).sum() == 0

# Target must be positive
df = df[df["district_atm_withdrawal_volume_mn"] > 0]

# Shares must sum ~1
df["atm_share_txn"] = df["atm_share_txn"].clip(0, 1)
df["digital_share_txn"] = df["digital_share_txn"].clip(0, 1)


In [ ]:
DROP_COLS = [
    "withdrawal_volume_million",        # national total
    "withdrawal_value_crore",
    "district_share",                   # derived from target
    "district_atm_withdrawal_value_cr", # correlated with volume
]

df = df.drop(columns=DROP_COLS, errors="ignore")


In [ ]:
DROP_COLS = [
    "withdrawal_volume_million",        # national total
    "withdrawal_value_crore",
    "district_share",                   # derived from target
    "district_atm_withdrawal_value_cr", # correlated with volume
]

df = df.drop(columns=DROP_COLS, errors="ignore")


In [ ]:
df["quarter"] = ((df["month_num"] - 1) // 3) + 1


In [ ]:
LAGS = [1, 2, 3, 6, 12]

for lag in LAGS:
    df[f"lag_{lag}"] = (
        df.groupby("district_name")["district_atm_withdrawal_volume_mn"]
          .shift(lag)
    )


In [ ]:
df["roll_mean_3"] = (
    df.groupby("district_name")["district_atm_withdrawal_volume_mn"]
      .shift(1).rolling(3).mean()
)

df["roll_std_3"] = (
    df.groupby("district_name")["district_atm_withdrawal_volume_mn"]
      .shift(1).rolling(3).std()
)


In [ ]:
df_model = df.dropna().reset_index(drop=True)


In [ ]:
print("Final shape:", df_model.shape)
print("Districts:", df_model["district_name"].nunique())
print("Months:", df_model["month"].nunique())
print("Target min/max:",
      df_model["district_atm_withdrawal_volume_mn"].min(),
      df_model["district_atm_withdrawal_volume_mn"].max())


Final shape: (88028, 32)
Districts: 746
Months: 118
Target min/max: 0.018524452 0.583537031


In [ ]:
df_model.to_csv("district_month_FINAL_MODEL_READY.csv", index=False)


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/district_month_FINAL_MODEL_READY.csv")
print(df.shape)


(88028, 32)


In [ ]:
REQUIRED = [
    "district_name", "year", "month_num",
    "district_atm_withdrawal_volume_mn"
]

for c in REQUIRED:
    assert c in df.columns, f"Missing column: {c}"


In [ ]:
df = df.sort_values(
    ["district_name", "year", "month_num"]
).reset_index(drop=True)


In [ ]:
LAGS = [1, 2, 3, 6, 12]

for lag in LAGS:
    df[f"lag_{lag}"] = (
        df.groupby("district_name")[TARGET]
        .shift(lag)
    )


In [ ]:
df["roll_mean_3"] = (
    df.groupby("district_name")[TARGET]
    .shift(1)
    .rolling(3)
    .mean()
)

df["roll_mean_6"] = (
    df.groupby("district_name")[TARGET]
    .shift(1)
    .rolling(6)
    .mean()
)

df["roll_std_6"] = (
    df.groupby("district_name")[TARGET]
    .shift(1)
    .rolling(6)
    .std()
)


In [ ]:
df = df.dropna().reset_index(drop=True)
print("After lag drop:", df.shape)


After lag drop: (79076, 34)


In [ ]:
DROP_COLS = [
    # identifiers
    "state_name",
    "district_name",

    # raw national values (leakage)
    "withdrawal_volume_million",
    "withdrawal_value_crore",

    # redundant splits
    "digital_txn_volume_mn",

    # time helpers (encoded already)
    "month"
]

DROP_COLS = [c for c in DROP_COLS if c in df.columns]
df = df.drop(columns=DROP_COLS)


In [ ]:
assert (df[TARGET] > 0).all(), "Target has non-positive values"
assert df.isna().sum().sum() == 0, "NaNs remain"

print("Final dataset shape:", df.shape)
df.head()


Final dataset shape: (79076, 30)


,year,month_num,num_atms,population_total_k,working_age_pct,literacy_rate,median_income_k,business_activity_index,cash_intensity_index,metro_atm_share,...,quarter,lag_1,lag_2,lag_3,lag_6,lag_12,roll_mean_3,roll_std_3,roll_mean_6,roll_std_6
0,2017,1,4633,2360.126796,52.237335,58.728891,21.30341,0.466125,0.809883,0.0,...,1,0.238085,0.282413,0.278193,0.203182,0.217262,0.266230,0.024466,0.238170,0.036083
1,2017,2,4633,2360.126796,52.237335,58.728891,21.30341,0.466125,0.809883,0.0,...,1,0.294757,0.238085,0.282413,0.197503,0.240135,0.271751,0.029802,0.253433,0.037656
2,2017,3,4633,2360.126796,52.237335,58.728891,21.30341,0.466125,0.809883,0.0,...,1,0.295230,0.294757,0.238085,0.229645,0.279612,0.276024,0.032857,0.269720,0.028696
3,2017,4,4633,2360.126796,52.237335,58.728891,21.30341,0.466125,0.809883,0.0,...,2,0.293297,0.295230,0.294757,0.278193,0.235166,0.294428,0.001007,0.280329,0.021872
4,2017,5,4633,2360.126796,52.237335,58.728891,21.30341,0.466125,0.809883,0.0,...,2,0.272040,0.293297,0.295230,0.282413,0.206732,0.286856,0.012867,0.279304,0.022134


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79076 entries, 0 to 79075
Data columns (total 30 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   year                               79076 non-null  int64  
 1   month_num                          79076 non-null  int64  
 2   num_atms                           79076 non-null  int64  
 3   population_total_k                 79076 non-null  float64
 4   working_age_pct                    79076 non-null  float64
 5   literacy_rate                      79076 non-null  float64
 6   median_income_k                    79076 non-null  float64
 7   business_activity_index            79076 non-null  float64
 8   cash_intensity_index               79076 non-null  float64
 9   metro_atm_share                    79076 non-null  float64
 10  urban_atm_share                    79076 non-null  float64
 11  rural_atm_share                    79076 non-null  flo

In [ ]:
df.to_csv(
    "/content/district_month_MODEL_READY_LAGS.csv",
    index=False
)
